**Load file with dialogue**

In [1]:
from tkinter.filedialog import askopenfilename
from tkinter import Tk

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename()
print(filename)


C:/Users/iveng/Downloads/example.csv


In [17]:
import csv

f = open(filename, 'r')

with f:
    reader = csv.reader(f)


### For now assume that there are no row labels and column labels are simply given in row 1

<function TextIOWrapper.close()>

**We're going to generate readable .ttl directly**
Future implementation would make it into proper rdflib objects and outputs that are flexible from a formatting perspective4

In [30]:
prefixes = """
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix csvw: <http://www.w3.org/ns/csvw#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix : <http://tables/>
@prefix : <http://schemas/>

@prefix csvw-ext: <http://csvw-ext#> .

csvw-ext a owl:Ontology .

csvw-ext:isValueInRowID a owl:DatatypeProperty;
    rdfs:label "is Value in Row ID";
    rdfs:comment "Relates a value to a certain row number in a table" ;
    rdfs:domain csvw:Cell ;
    rdfs:range xsd:integer ;
    .

csvw-ext:isValueInColumnName a owl:DatatypeProperty;
    rdfs:label "is Value in Column Name";
    rdfs:comment "Relates a value to a certain column name in a table" ;
    rdfs:domain csvw:Cell ;
    rdfs:range xsd:string ;
    .
    
    
    


"""

Where we want to get to is something like this (assuming first row and COLUMN_LABEL1 and COLUMN_LABEL2 in TABLE_X and creating SCHEMA_X with values in 1:1 given as 'value_in_cell_11')


This follows with some deviation the guidance on:
(https://www.w3.org/TR/2015/REC-csv2rdf-20151217/)



##### Once per table group
```sparql
_:G a csvw:TableGroup .

```

##### Once for the table
```sparql
_:T csvw:table _:G .       ## put the table in the TableGroup
_:T a csvw:Table .         ## declares the table and puts it in a namespace
_:T csvw:url :TABLE_X .    ## gives the table a URL

:SCHEMA_X a csvw:Schema         ## declares the schema we're going to be building out
:TABLE_X csvw:tableSchema :SCHEMA_X  # I'd make the object property something like hasTableSchema; rather than tableSchema
```

##### Go over each Column to build out the basic schema
```sparql
_:column1 a csvw:Column .             ## Declare Column
_:column1 csvw:column :SCHEMA_X .     ## Declare that the column forms part of the SCHEMA we are building
_:column1 csvw:name "COLUMN_LABEL1" . ## Provide the canonical name for the column 

_:column2 a csvw:Column .
_:column2 csvw:column :SCHEMA_X .      
_:column2 csvw:name "COLUMN_LABEL2" .
# etc.
```

##### For each row in the table

```sparql
_:row1 csvw:row :TABLE_X .         ## Denotes that the row is in the Table
_:row1 a csvw:Row .                ## declares the row
_:row1 csvw:rownum 1^xsd:integer . ## marks the row number (start with 1)

```

##### For each cell at the intersection of the rows and columns (e.g. cell in row 1, column "Column 1")
```sparql
_:cell11 a csvw:Cell .
_:cell11 csvw-ext:isValueInRowID 1 .
_:cell11 csvw-ext:isValueInColumnName "COLUMN_LABEL1" .
_:cell11 csvw:describes "value_in_cell_11" 

_:cell11 csvw:name "COLUMN_LABEL1" .
_:row1 csvw:describes "value_in_cell_11" .

_:cell11 




In [28]:
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF





g = Graph()

# Create an identifier to use as the subject for Donna.
donna = BNode()

# Add triples using store's add method.
g.add( (donna, RDF.type, FOAF.Person) )
g.add( (donna, FOAF.nick, Literal("donna", lang="foo")) )
g.add( (donna, FOAF.name, Literal("Donna Fales")) )
g.add( (donna, FOAF.mbox, URIRef("mailto:donna@example.org")) )

# Iterate over triples in store and print them out.
print("--- printing raw triples ---")
for s, p, o in g:
    print((s, p, o))

--- printing raw triples ---
(rdflib.term.BNode('N4233d509a4d94c79ba21446e1772dbe7'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/nick'), rdflib.term.Literal('donna', lang='foo'))
(rdflib.term.BNode('N4233d509a4d94c79ba21446e1772dbe7'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/name'), rdflib.term.Literal('Donna Fales'))
(rdflib.term.BNode('N4233d509a4d94c79ba21446e1772dbe7'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Person'))
(rdflib.term.BNode('N4233d509a4d94c79ba21446e1772dbe7'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/mbox'), rdflib.term.URIRef('mailto:donna@example.org'))


In [18]:


csvw = Namespace('http://www.w3.org/ns/csvw#')




import rdflib




fout = open("filename"+".ttl", 'w') 
# Default serialization is .ttl

@prefix csvw: <http://www.w3.org/ns/csvw#> .



In [3]:
from rdflib import URIRef
from rdflib.namespace import Namespace

from csvwlib.utils.rdf import CSVW


class Namespaces:
    """ RDFa initial context
    https://www.w3.org/2011/rdfa-context/rdfa-1.1
    """

    @staticmethod
    def all():
        return {
            # Vocabulary Prefixes of W3C Documents
            'as': Namespace('https://www.w3.org/ns/activitystreams#'),
            'csvw': Namespace('http://www.w3.org/ns/csvw#'),
            'dcat': Namespace('http://www.w3.org/ns/dcat#'),
            'dqv': Namespace('http://www.w3.org/ns/dqv#'),
            'duv': Namespace('https://www.w3.org/TR/vocab-duv#'),
            'grddl': Namespace('http://www.w3.org/2003/g/data-view#'),
            'ldp': Namespace('http://www.w3.org/ns/ldp#'),
            'ma': Namespace('http://www.w3.org/ns/ma-ont#'),
            'oa': Namespace('http://www.w3.org/ns/oa#'),
            'org': Namespace('http://www.w3.org/ns/org#'),
            'owl': Namespace('http://www.w3.org/2002/07/owl#'),
            'prov': Namespace('http://www.w3.org/ns/prov#'),
            'qb': Namespace('http://purl.org/linked-data/cube#'),
            'rdf': Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#'),
            'rdfa': Namespace('http://www.w3.org/ns/rdfa#'),
            'rdfs': Namespace('http://www.w3.org/2000/01/rdf-schema#'),
            'rif': Namespace('http://www.w3.org/2007/rif#'),
            'rr': Namespace('http://www.w3.org/ns/r2rml#'),
            'sd': Namespace('http://www.w3.org/ns/sparql-service-description#'),
            'skos': Namespace('http://www.w3.org/2004/02/skos/core#'),
            'skosxl': Namespace('http://www.w3.org/2008/05/skos-xl#'),
            'ssn': Namespace('http://www.w3.org/ns/ssn/'),
            'sosa': Namespace('http://www.w3.org/ns/sosa/'),
            'time': Namespace('http://www.w3.org/2006/time#'),
            'void': Namespace('http://rdfs.org/ns/void#'),
            'wdr': Namespace('http://www.w3.org/2007/05/powder#'),
            'wdrs': Namespace('http://www.w3.org/2007/05/powder-s#'),
            'xhv': Namespace('http://www.w3.org/1999/xhtml/vocab#'),
            'xml': Namespace('http://www.w3.org/XML/1998/namespace'),
            'xsd': Namespace('http://www.w3.org/2001/XMLSchema#'),
            # Widely used Vocabulary prefixes
            'cc': Namespace('http://creativecommons.org/ns#'),
            'ctag': Namespace('http://commontag.org/ns#'),
            'dc': Namespace('http://purl.org/dc/terms/'),
            'dcterms': Namespace('http://purl.org/dc/terms/'),
            'dc11': Namespace('http://purl.org/dc/elements/1.1/'),
            'foaf': Namespace('http://xmlns.com/foaf/0.1/'),
            'gr': Namespace('http://purl.org/goodrelations/v1#'),
            'ical': Namespace('http://www.w3.org/2002/12/cal/icaltzd#'),
            'og': Namespace('http://ogp.me/ns#'),
            'rev': Namespace('http://purl.org/stuff/rev#'),
            'sioc': Namespace('http://rdfs.org/sioc/ns#'),
            'v': Namespace('http://rdf.data-vocabulary.org/#'),
            'vcard': Namespace('http://www.w3.org/2006/vcard/ns#'),
            'schema': Namespace('http://schema.org/'),
        }

    @staticmethod
    def get(prefix):
        return Namespaces.all().get(prefix)

    @staticmethod
    def get_term(string):
        if string in CSVW.term_mappings.keys():
            uri = CSVW.term_mappings[string]
            if uri.startswith('http'):
                return URIRef(uri)
            return Namespaces.get_term(uri)
        if string == '@type':
            return Namespaces.get('rdf').type
        prefix, term = string.split(':', 1) if string is not None else ('', '')
        return Namespaces.get(prefix).term(term) if prefix in Namespaces.all() else None

    @staticmethod
    def replace_url_with_prefix(url):
        """ Replaces absolute URL with prefix:term if domain is in known
        namespaces """
        for prefix, namespace in Namespaces.all().items():
            if url.startswith(namespace):
                return url.replace(namespace, prefix + ':')
        return url

@prefix : <https://raw.githubusercontent.com/ivenger/RDF-parser-normalizer/master/example.csv#> .
@prefix as: <https://www.w3.org/ns/activitystreams#> .
@prefix cc: <http://creativecommons.org/ns#> .
@prefix csvw: <http://www.w3.org/ns/csvw#> .
@prefix ctag: <http://commontag.org/ns#> .
@prefix dc: <http://purl.org/dc/terms/> .
@prefix dc11: <http://purl.org/dc/elements/1.1/> .
@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix dqv: <http://www.w3.org/ns/dqv#> .
@prefix duv: <https://www.w3.org/TR/vocab-duv#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix gr: <http://purl.org/goodrelations/v1#> .
@prefix grddl: <http://www.w3.org/2003/g/data-view#> .
@prefix ical: <http://www.w3.org/2002/12/cal/icaltzd#> .
@prefix ldp: <http://www.w3.org/ns/ldp#> .
@prefix ma: <http://www.w3.org/ns/ma-ont#> .
@prefix ns1: <https://raw.githubusercontent.com/ivenger/RDF-parser-normalizer/master/example.csv#%EF%BB%> .
@prefix ns2: <https://raw.githu